<a href="https://colab.research.google.com/github/swetabehera04/Horse_Human_Classifer_using_CNN/blob/master/Horse_Human_Classifier_using_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Scratch ResNet Image Classification


In [1]:
# !pip install tensorflow==2.0.0
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.python.keras.layers import *
import six
from tensorflow.python.keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.layers import Dense, Conv2D, BatchNormalization, Activation
from tensorflow.python.keras.layers import AveragePooling2D, Input, Flatten
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.python.keras.callbacks import ReduceLROnPlateau
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.regularizers import l2
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import *
from tensorflow.python.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.utils.conv_utils import convert_kernel
from tensorflow.python.util import nest
from tensorflow.python.util import object_identity
from tensorflow.python.util.tf_export import keras_export
import zipfile


In [2]:
class DefaultConfig():

    try:
        model_name = sys.argv[1]
    except:
        print("use default model LeNet, see config.py")
        model_name = "LeNet"

    train_data_path = './dataset/train/'
    test_data_path = './dataset/test/'
    checkpoints = './checkpoints/'

    if model_name == 'InceptionV3':
        normal_size = 75#minSize
    elif model_name == 'Xception':
        normal_size = 71#minSize
    else:
        normal_size = 28
    # normal_size = 48
    epochs = 1
    batch_size = 2
    classNumber = 10 # see dataset/tri
    channles = 1 # or 3 or 1
    lr = 0.00001
    lr_reduce_patience = 5  # 需要降低学习率的训练步长
    early_stop_patience = 11  # 提前终止训练的步长

    data_augmentation = False
    monitor = 'val_loss'
    cut = False
    rat = 0.1 #if cut,img[slice(h*self.rat,h-h*self.rat),slice(w*self.rat,w-w*self.rat)]
config=DefaultConfig()

use default model LeNet, see config.py


In [3]:
class MODEL(object):

    def __init__(self,config):
        self.config = config

    def input_shape_define(self):
        return  (self.config.normal_size, self.config.normal_size, self.config.channles)

    def covn_block(self,model,kenal_number,kenal_size,padding,activation):
        model.add(Convolution2D(kenal_number,kenal_size,padding=padding,activation=activation))
        return model

    def max_pooling_type(self,model,kenal_size,strides):
        model.add(MaxPooling2D(pool_size=kenal_size,strides=strides))
        return model

    def mnist_net(self):
        model = Sequential()
        input_shape = (self.config.normal_size, self.config.normal_size, self.config.channles)
        model.add(Convolution2D(96,(3,3),input_shape=input_shape,padding='same',activation='relu',kernel_initializer='uniform'))
        model.add(Convolution2D(128,(3,3),padding='same',activation='relu'))
        model.add(Convolution2D(128,(1,1),padding='same',activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        model.add(Convolution2D(256,(3,3),padding='same',activation='relu'))
        model.add(Convolution2D(256,(1,1),padding='same',activation='relu'))
        model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
        model.add(Convolution2D(512, (3, 3), padding='same', activation='relu'))
        model.add(Convolution2D(512, (3, 3), padding='same', activation='relu'))
        model.add(Convolution2D(256, (1, 1), padding='same', activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

        model.add(Convolution2D(512, (3, 3), padding='same', activation='relu'))
        model.add(Convolution2D(512, (3, 3), padding='same', activation='relu'))
        model.add(Convolution2D(256, (1, 1), padding='same', activation='relu'))
        # model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

        model.add(Flatten())
        model.add(Dense(4096,activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(2048, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(self.config.classNumber,activation='softmax'))
        return model

    #VGG16
    def TSL16(self):
        model = Sequential()
        input_shape = (self.config.normal_size, self.config.normal_size, self.config.channles)
        model.add(Convolution2D(64,kernel_size=(3,3),input_shape=input_shape,padding='same',activation='relu'))
        model.add(Convolution2D(64,kernel_size=(3,3),padding='same',activation='relu'))
        model = self.max_pooling_type(model,kenal_size=(2,2),strides=(2,2))
        for i in range(2):
            model = self.covn_block(model, kenal_number=128, kenal_size=(3, 3), padding='same', activation='relu')

        model = self.max_pooling_type(model,kenal_size = (2,2),strides=(2,2))
        for i in range(3):
            model = self.covn_block(model,kenal_number=128,  kenal_size=(3,3),  padding='same',activation='relu')

        model = self.max_pooling_type(model,kenal_size=(2,2),strides=(2,2))
        for i in range(3):
            model = self.covn_block(model,kenal_number=512,kenal_size=(3,3),padding='same',activation='relu')

        model = self.max_pooling_type(model,kenal_size=(2,2),strides=(2,2))
        for i in range(3):
            model = self.covn_block(model,kenal_number=512,kenal_size=(3,3),padding='same',activation='relu')

        model.add(Flatten())
        model.add(Dense(4096,activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(1024,activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(self.config.classNumber,activation='softmax'))
        return model


    # AlexNet
    def AlexNet(self):
        model = Sequential()
        # input_shape = (64,64, self.config.channles)
        input_shape = (224, 224, 1)
        model.add(Convolution2D(96, (11, 11), input_shape=input_shape,strides=(4, 4),  padding='valid',activation='relu', kernel_initializer='uniform'))
        model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))#26*26
        model.add(Convolution2D(256, (5, 5), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform'))
        model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
        model.add(Convolution2D(384, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform'))
        model.add(Convolution2D(384, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform'))
        model.add(Convolution2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform'))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        model.add(Flatten())
        model.add(Dense(4096, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(4096, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(self.config.classNumber, activation='softmax'))
        # model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
        return model

    def VGG16(self):
        model = Sequential()
        input_shape= (self.config.normal_size, self.config.normal_size, self.config.channles)
        model.add(Convolution2D(64,(3,3),input_shape=input_shape,activation='relu',padding='same'))
        model.add(Convolution2D(64,(3,3),activation='relu',padding='same'))
        model.add(Convolution2D(128,(3,3),activation='relu',padding='same'))
        model.add(Convolution2D(128,(3,3),activation='relu',padding='same'))
        model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
        model.add(Convolution2D(256,(3,3),activation='relu',padding='same'))
        model.add(Convolution2D(256,(3,3),activation='relu',padding='same'))
        model.add(Convolution2D(256,(3,3),activation='relu',padding='same'))
        model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
        model.add(Convolution2D(512,(3,3),activation='relu',padding='same'))
        model.add(Convolution2D(512,(3,3),activation='relu',padding='same'))
        model.add(Convolution2D(512,(3,3),activation='relu',padding='same'))
        model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
        model.add(Convolution2D(512,(3,3),activation='relu',padding='same'))
        model.add(Convolution2D(512,(3,3),activation='relu',padding='same'))
        model.add(Convolution2D(512,(3,3),activation='relu',padding='same'))
        model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
        model.add(Flatten())
        model.add(Dense(4096,activation='relu'))
        model.add(Dense(4096,activation='relu'))
        model.add(Dense(self.config.classNumber,activation='softmax'))
        return model

    def VGG19(self):
        model = Sequential()
        input_shape= (self.config.normal_size, self.config.normal_size, self.config.channles)
        model.add(Convolution2D(64,(3,3),input_shape=input_shape,activation='relu',padding='same'))
        model.add(Convolution2D(64,(3,3),activation='relu',padding='same'))
        model.add(Convolution2D(128,(3,3),activation='relu',padding='same'))
        model.add(Convolution2D(128,(3,3),activation='relu',padding='same'))
        model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
        model.add(Convolution2D(256,(3,3),activation='relu',padding='same'))
        model.add(Convolution2D(256,(3,3),activation='relu',padding='same'))
        model.add(Convolution2D(256,(3,3),activation='relu',padding='same'))
        model.add(Convolution2D(256,(3,3),activation='relu',padding='same'))
        model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
        model.add(Convolution2D(512,(3,3),activation='relu',padding='same'))
        model.add(Convolution2D(512,(3,3),activation='relu',padding='same'))
        model.add(Convolution2D(512,(3,3),activation='relu',padding='same'))
        model.add(Convolution2D(512,(3,3),activation='relu',padding='same'))
        model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
        model.add(Convolution2D(512,(3,3),activation='relu',padding='same'))
        model.add(Convolution2D(512,(3,3),activation='relu',padding='same'))
        model.add(Convolution2D(512,(3,3),activation='relu',padding='same'))
        model.add(Convolution2D(512,(3,3),activation='relu',padding='same'))
        model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
        model.add(Flatten())
        model.add(Dense(4096,activation='relu'))
        model.add(Dense(4096,activation='relu'))
        model.add(Dense(self.config.classNumber,activation='softmax'))
        return model

    #LeNet
    def LeNet(self):
        # initialize the model
        model = Sequential()
        inputShape = (self.config.normal_size, self.config.normal_size, self.config.channles)
        model.add(Conv2D(20, (5, 5), padding="same", input_shape=inputShape))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        # second set of CONV => RELU => POOL layers
        model.add(Conv2D(50, (5, 5), padding="same"))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        # first (and only) set of FC => RELU layers
        model.add(Flatten())
        model.add(Dense(500))
        model.add(Activation("relu"))

        # softmax classifier
        model.add(Dense(self.config.classNumber))
        model.add(Activation("softmax"))
        # model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
        # return the constructed network architecture
        return model

    #ZF_Net,8 layers
    def ZF_Net(self):
        model = Sequential()
        model.add(
            Conv2D(96, (7, 7), strides=(2, 2),
                   input_shape=(224, 224,1),
                   padding='valid',
                   activation='relu',
                   kernel_initializer='uniform'))
        model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
        model.add(Conv2D(256, (5, 5), strides=(2, 2), padding='same', activation='relu', kernel_initializer='uniform'))
        model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
        model.add(Conv2D(384, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform'))
        model.add(Conv2D(384, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform'))
        model.add(Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='uniform'))
        model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
        model.add(Flatten())
        model.add(Dense(4096, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(4096, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(self.config.classNumber, activation='softmax'))
        # model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
        return model

config=DefaultConfig()
mdbd = MODEL(config)
lenet = mdbd.LeNet()

vgg16 = mdbd.VGG16()
vgg19 = mdbd.VGG19()
zfnet = mdbd.ZF_Net()
alexnet = mdbd.AlexNet()




Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


{'alexnet(224x224x1)': Dimension(204608),
 'lenet(28x28x1)': Dimension(7390),
 'vgg16(28x28x1)': Dimension(266176),
 'vgg19(28x28x1)': Dimension(308416),
 'zfnet(224x224x1)': Dimension(431936)}

In [0]:
class ResnetBuilder(object):
    # @staticmethod
    def build(self, block_fn, repetitions):
        """Builds a custom ResNet like architecture.
        Args:
            input_shape: The input shape in the form (nb_channels, nb_rows, nb_cols)
            num_outputs: The number of outputs at final softmax layer
            block_fn: The block function to use. This is either `basic_block` or `bottleneck`.
                The original paper used basic_block for layers < 50
            repetitions: Number of repetitions of various block units.
                At each block unit, the number of filters are doubled and the input size is halved
        Returns:
            The keras `Model`.
        """

        input_shape = (28,28,1)
        num_outputs = 10
        self._handle_dim_ordering()
        block_fn = self._get_block(block_fn)

        input = Input(shape=input_shape)
        conv1 = self._conv_bn_relu(filters=64, kernel_size=(7, 7), strides=(2, 2))(input)
        pool1 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding="same")(conv1)

        block = pool1
        filters = 64
        for i, r in enumerate(repetitions):
            block = self._residual_block(block_fn, filters=filters, repetitions=r, is_first_layer=(i == 0))(block)
            filters *= 2

        # Last activation
        block = self._bn_relu(block)

        # Classifier block
        block_shape = K.int_shape(block)
        pool2 = AveragePooling2D(pool_size=(block_shape[ROW_AXIS], block_shape[COL_AXIS]),
                                 strides=(1, 1))(block)

        flatten1 = Flatten()(pool2)
        dense = Dense(units=num_outputs,
                      activation="softmax")(flatten1)

        model = Model(inputs=input, outputs=dense)
        # model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
        return model

    # @staticmethod
    def build_custom_resnet(self,params):
        return self.build(self.basic_block, [params[0], params[1], params[2], params[3]])
        
    # @staticmethod
    def build_resnet18(self):
        return self.build(self.basic_block, [2, 2, 2, 2])

    # @staticmethod
    def build_resnet34(self):
        return self.build( self.basic_block, [3, 4, 6, 3])

    # @staticmethod
    def build_resnet50(self):
        return self.build( self.bottleneck, [3, 4, 6, 3])

    # @staticmethod
    def build_resnet101(self):
        return self.build(self.bottleneck, [3, 4, 23, 3])

    # @staticmethod
    def build_resnet152(self):
        return self.build( self.bottleneck, [3, 8, 36, 3])

    def _bn_relu(self,input):
        """Helper to build a BN -> relu block
        """
        norm = BatchNormalization(axis=CHANNEL_AXIS)(input)
        return Activation("relu")(norm)

    def _conv_bn_relu(self,**conv_params):
        """Helper to build a conv -> BN -> relu block
        """
        filters = conv_params["filters"]
        kernel_size = conv_params["kernel_size"]
        strides = conv_params.setdefault("strides", (1, 1))
        kernel_initializer = conv_params.setdefault("kernel_initializer", "he_normal")
        padding = conv_params.setdefault("padding", "same")
        kernel_regularizer = conv_params.setdefault("kernel_regularizer", l2(1.e-4))

        def f(input):
            conv = Conv2D(filters=filters, kernel_size=kernel_size,
                          strides=strides, padding=padding,
                          kernel_initializer=kernel_initializer,
                          kernel_regularizer=kernel_regularizer)(input)
            return self._bn_relu(conv)

        return f


    def _bn_relu_conv(self,**conv_params):
        """Helper to build a BN -> relu -> conv block.
        This is an improved scheme proposed in http://arxiv.org/pdf/1603.05027v2.pdf
        """
        filters = conv_params["filters"]
        kernel_size = conv_params["kernel_size"]
        strides = conv_params.setdefault("strides", (1, 1))
        kernel_initializer = conv_params.setdefault("kernel_initializer", "he_normal")
        padding = conv_params.setdefault("padding", "same")
        kernel_regularizer = conv_params.setdefault("kernel_regularizer", l2(1.e-4))

        def f(input):
            activation = self._bn_relu(input)
            return Conv2D(filters=filters, kernel_size=kernel_size,
                          strides=strides, padding=padding,
                          kernel_initializer=kernel_initializer,
                          kernel_regularizer=kernel_regularizer)(activation)

        return f


    def _shortcut(self,input, residual):
        """Adds a shortcut between input and residual block and merges them with "sum"
        """
        # Expand channles of shortcut to match residual.
        # Stride appropriately to match residual (width, height)
        # Should be int if network architecture is correctly configured.
        input_shape = K.int_shape(input)
        residual_shape = K.int_shape(residual)
        stride_width = int(round(input_shape[ROW_AXIS] / residual_shape[ROW_AXIS]))
        stride_height = int(round(input_shape[COL_AXIS] / residual_shape[COL_AXIS]))
        equal_channels = input_shape[CHANNEL_AXIS] == residual_shape[CHANNEL_AXIS]

        shortcut = input
        # 1 X 1 conv if shape is different. Else identity.
        if stride_width > 1 or stride_height > 1 or not equal_channels:
            shortcut = Conv2D(filters=residual_shape[CHANNEL_AXIS],
                              kernel_size=(1, 1),
                              strides=(stride_width, stride_height),
                              padding="valid",
                              kernel_initializer="he_normal",
                              kernel_regularizer=l2(0.0001))(input)

        return add([shortcut, residual])


    def _residual_block(self,block_function, filters, repetitions, is_first_layer=False):
        """Builds a residual block with repeating bottleneck blocks.
        """
        def f(input):
            for i in range(repetitions):
                init_strides = (1, 1)
                if i == 0 and not is_first_layer:
                    init_strides = (2, 2)
                input = block_function(filters=filters, init_strides=init_strides,
                                       is_first_block_of_first_layer=(is_first_layer and i == 0))(input)
            return input

        return f


    def basic_block(self,filters, init_strides=(1, 1), is_first_block_of_first_layer=False):
        """Basic 3 X 3 convolution blocks for use on resnets with layers <= 34.
        Follows improved proposed scheme in http://arxiv.org/pdf/1603.05027v2.pdf
        """
        def f(input):

            if is_first_block_of_first_layer:
                # don't repeat bn->relu since we just did bn->relu->maxpool
                conv1 = Conv2D(filters=filters, kernel_size=(3, 3),
                               strides=init_strides,
                               padding="same",
                               kernel_initializer="he_normal",
                               kernel_regularizer=l2(1e-4))(input)
            else:
                conv1 = self._bn_relu_conv(filters=filters, kernel_size=(3, 3),
                                      strides=init_strides)(input)

            residual = self._bn_relu_conv(filters=filters, kernel_size=(3, 3))(conv1)
            return self._shortcut(input, residual)

        return f


    def bottleneck(self,filters, init_strides=(1, 1), is_first_block_of_first_layer=False):
        """Bottleneck architecture for > 34 layer resnet.
        Follows improved proposed scheme in http://arxiv.org/pdf/1603.05027v2.pdf
        Returns:
            A final conv layer of filters * 4
        """
        def f(input):

            if is_first_block_of_first_layer:
                # don't repeat bn->relu since we just did bn->relu->maxpool
                conv_1_1 = Conv2D(filters=filters, kernel_size=(1, 1),
                                  strides=init_strides,
                                  padding="same",
                                  kernel_initializer="he_normal",
                                  kernel_regularizer=l2(1e-4))(input)
            else:
                conv_1_1 = self._bn_relu_conv(filters=filters, kernel_size=(1, 1),
                                         strides=init_strides)(input)

            conv_3_3 = self._bn_relu_conv(filters=filters, kernel_size=(3, 3))(conv_1_1)
            residual = self._bn_relu_conv(filters=filters * 4, kernel_size=(1, 1))(conv_3_3)
            return self._shortcut(input, residual)

        return f

    def _handle_dim_ordering(self):
        global ROW_AXIS
        global COL_AXIS
        global CHANNEL_AXIS
    
        ROW_AXIS = 1
        COL_AXIS = 2
        CHANNEL_AXIS = 3
      

    def _get_block(self,identifier):
        if isinstance(identifier, six.string_types):
            res = globals().get(identifier)
            if not res:
                raise ValueError('Invalid {}'.format(identifier))
            return res
        return identifier
resnetbd = ResnetBuilder()
resnet2232 = resnetbd.build_custom_resnet([2,2,3,2])
resnet1232 = resnetbd.build_custom_resnet([1,2,3,2])
resnet1222 = resnetbd.build_custom_resnet([1,2,2,2])
resnet1212 = resnetbd.build_custom_resnet([1,2,1,2])
resnet2111 = resnetbd.build_custom_resnet([2,1,1,1])
resnet1112 = resnetbd.build_custom_resnet([1,1,1,2])
resnet1111 = resnetbd.build_custom_resnet([1,1,1,1])
resnet3232 = resnetbd.build_custom_resnet([3,2,3,2])
resnet3463 = resnetbd.build_custom_resnet([3,4,6,3])
resnet2222 = resnetbd.build_custom_resnet([2,2,2,2]) #18
resnet50 = resnetbd.build_resnet50() #3463
resnet152 =resnetbd.build_resnet152()#3,8,36,3
resnet101 =resnetbd.build_resnet101()#3,8,36,3


In [0]:
def capacity_estimate_v2(model):
  capacity_dic = {}
  capacity_list = []
  output_dic={}
  
  MAX_VALUE=10000000
  # pre_capcity=MAX_VALUE
  bottle_neck_capacity = MAX_VALUE
  total_capacity = 0
  for layer in model.layers:
    connections=[]
    for node in layer._inbound_nodes:
      for inbound_layer, node_index, tensor_index, _ in node.iterate_inbound():
        connections.append(inbound_layer.name)
      
    capacity=0  
    if len(connections)!=0:
      for layer_name in connections:
        if layer_name.find('input')!=-1:
          capacity = model.input.shape[1]*model.input.shape[2]*model.input.shape[3]  
        else:
          capacity = max(output_dic[layer_name],capacity)
    else:
        capacity = MAX_VALUE
    
    capacity = min(capacity,layer.count_params())
    capacity_dic[layer.name] = min(bottle_neck_capacity,capacity)
    capacity_list.append((layer.name,min(bottle_neck_capacity,capacity)))

    # if capacity!=0:
    #   capacity_dic[layer.name] = min(pre_capcity,capacity)
    #   capacity_list.append((layer.name,min(pre_capcity,capacity)))
    #   pre_capcity=min(pre_capcity,capacity)
    # else:
    #   capacity_dic[layer.name] = capacity
    #   capacity_list.append((layer.name,capacity))
    output_bits = 0
    output_shape = layer.output.shape
    
    if len(output_shape) == 2:
      output_bits = output_shape[1]
    else:
      output_bits = output_shape[1]*output_shape[2]*output_shape[3]

    bottle_neck_capacity = min(output_bits,bottle_neck_capacity)

    output_dic[layer.name] = bottle_neck_capacity
    
    total_capacity += capacity

  return total_capacity,capacity_list
  
  

In [6]:
cpdic={}

cpdic['2232']=capacity_estimate_v2(resnet2232)[0]
cpdic['1232']=capacity_estimate_v2(resnet1232)[0]
cpdic['1222']=capacity_estimate_v2(resnet1222)[0]
cpdic['1212']=capacity_estimate_v2(resnet1212)[0]
cpdic['2111']=capacity_estimate_v2(resnet2111)[0]
cpdic['1112']=capacity_estimate_v2(resnet1112)[0]
cpdic['1111']=capacity_estimate_v2(resnet1111)[0]
cpdic['3232']=capacity_estimate_v2(resnet3232)[0]
cpdic['3463']=capacity_estimate_v2(resnet3463)[0]
cpdic['2222']=capacity_estimate_v2(resnet2222)[0]
cpdic['3463(50)']=capacity_estimate_v2(resnet50)[0]
cpdic['3463(50)']=capacity_estimate_v2(resnet50)[0]
cpdic['resnet151']=capacity_estimate_v2(resnet152)[0]
cpdic['resnet101']=capacity_estimate_v2(resnet101)[0]
cpdic

{'1111': Dimension(14032),
 '1112': Dimension(16080),
 '1212': Dimension(18672),
 '1222': Dimension(21808),
 '1232': Dimension(24944),
 '2111': Dimension(16112),
 '2222': Dimension(23888),
 '2232': Dimension(27024),
 '3232': Dimension(29104),
 '3463': Dimension(45744),
 '3463(50)': Dimension(69984),
 'resnet101': Dimension(149952),
 'resnet151': Dimension(227744)}

In [7]:
othernet={}
othernet['lenet(28x28x1)']=capacity_estimate_v2(lenet)[0]
othernet['vgg16(28x28x1)']=capacity_estimate_v2(vgg16)[0]
othernet['vgg19(28x28x1)']=capacity_estimate_v2(vgg19)[0]
othernet['zfnet(224x224x1)']=capacity_estimate_v2(zfnet)[0]
othernet['alexnet(224x224x1)']=capacity_estimate_v2(alexnet)[0]
othernet

{'alexnet(224x224x1)': Dimension(204608),
 'lenet(28x28x1)': Dimension(7390),
 'vgg16(28x28x1)': Dimension(266176),
 'vgg19(28x28x1)': Dimension(308416),
 'zfnet(224x224x1)': Dimension(431936)}

In [0]:
def count_params(weights):
  """Count the total number of scalars composing the weights.
  Arguments:
      weights: An iterable containing the weights on which to compute params
  Returns:
      The total number of scalars composing the weights
  """
  return int(
      sum(
          np.prod(p.shape.as_list())
          for p in object_identity.ObjectIdentitySet(weights)))
  
def print_summary(model, line_length=None, positions=None, print_fn=None):
    """Prints a summary of a model.
    # Arguments
        model: Keras model instance.
        line_length: Total length of printed lines
            (e.g. set this to adapt the display to different
            terminal window sizes).
        positions: Relative or absolute positions of log elements in each line.
            If not provided, defaults to `[.33, .55, .67, 1.]`.
        print_fn: Print function to use.
            It will be called on each line of the summary.
            You can set it to a custom function
            in order to capture the string summary.
            It defaults to `print` (prints to stdout).
    """
    if print_fn is None:
        print_fn = print

    if model.__class__.__name__ == 'Sequential':
        sequential_like = True
    elif not model._is_graph_network:
        # We treat subclassed models as a simple sequence of layers,
        # for logging purposes.
        sequential_like = True
    else:
        sequential_like = True
        nodes_by_depth = model._nodes_by_depth.values()
        nodes = []
        
        if sequential_like:
            # search for shared layers
            for layer in model.layers:
                flag = False
                for node in layer._inbound_nodes:
                    if node in nodes:
                        if flag:
                            sequential_like = False
                            break
                        else:
                            flag = True
                if not sequential_like:
                    break
    sequential_like=False

    if sequential_like:
        line_length = line_length or 65
        positions = positions or [.45, .85, 1.]
        if positions[-1] <= 1:
            positions = [int(line_length * p) for p in positions]
        # header names for the different log elements
        to_display = ['Layer (type)', 'Output Shape', 'Param #']
    else:
        line_length = line_length or 98
        positions = positions or [.33, .55, .67, 1.]
        if positions[-1] <= 1:
            positions = [int(line_length * p) for p in positions]
        # header names for the different log elements
        to_display = ['Layer (type)',
                      'Output Shape',
                      'Param #',
                      'Connected to']
        

    def print_row(fields, positions):
        line = ''
        for i in range(len(fields)):
            if i > 0:
                line = line[:-1] + ' '
            line += str(fields[i])
            line = line[:positions[i]]
            line += ' ' * (positions[i] - len(line))
        print_fn(line)

    print_fn('Model: "{}"'.format(model.name))
    print_fn('_' * line_length)
    print_row(to_display, positions)
    print_fn('=' * line_length)

    def print_layer_summary(layer):
        try:
            output_shape = layer.output_shape
        except AttributeError:
            output_shape = 'multiple'
        name = layer.name
        cls_name = layer.__class__.__name__
        fields = [name + ' (' + cls_name + ')',
                  output_shape, layer.count_params()]
        print_row(fields, positions)

    def print_layer_summary_with_connections(layer):
        """Prints a summary for a single layer.
        # Arguments
            layer: target layer.
        """
        try:
            output_shape = layer.output_shape
        except AttributeError:
            output_shape = 'multiple'
        connections = []
        for node in layer._inbound_nodes:
           
            # for i in range(len(node.inbound_layers)):
            #     inbound_layer = node.inbound_layers[i].name
            #     inbound_node_index = node.node_indices[i]
            #     inbound_tensor_index = node.tensor_indices[i]
            #     connections.append(inbound_layer +
            #                        '[' + str(inbound_node_index) + '][' +
            #                        str(inbound_tensor_index) + ']')
                
            for inbound_layer, node_index, tensor_index, _ in node.iterate_inbound():
                connections.append('{}[{}][{}]'.format(inbound_layer.name, node_index,
                                               tensor_index))

        name = layer.name
        cls_name = layer.__class__.__name__
        if not connections:
            first_connection = ''
        else:
            first_connection = connections[0]
        fields = [name +
                  ' (' + cls_name + ')',
                  output_shape,
                  layer.count_params(),
                  first_connection]
        print_row(fields, positions)
        if len(connections) > 1:
            for i in range(1, len(connections)):
                fields = ['', '', '', connections[i]]
                print_row(fields, positions)

    layers = model.layers
    for i in range(len(layers)):
        if sequential_like:
            print_layer_summary(layers[i])
        else:
            print_layer_summary_with_connections(layers[i])
        if i == len(layers) - 1:
            print_fn('=' * line_length)
        else:
            print_fn('_' * line_length)

    model._check_trainable_weights_consistency()
    if hasattr(model, '_collected_trainable_weights'):
        trainable_count = count_params(model._collected_trainable_weights)
    else:
        trainable_count = count_params(model.trainable_weights)

    non_trainable_count = count_params(model.non_trainable_weights)

    
    print_fn(
        'Total params: {:,}'.format(trainable_count + non_trainable_count))
    print_fn('Trainable params: {:,}'.format(trainable_count))
    print_fn('Non-trainable params: {:,}'.format(non_trainable_count))
    print_fn('_' * line_length)

In [9]:
print_summary(resnet1111)

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_145 (Conv2D)             (None, 14, 14, 64)   3200        input_7[0][0]                    
__________________________________________________________________________________________________
batch_normalization_86 (BatchNo (None, 14, 14, 64)   256         conv2d_145[0][0]                 
__________________________________________________________________________________________________
activation_90 (Activation)      (None, 14, 14, 64)   0           batch_normalization_86[0][0]     
____________________________________________________________________________________________